In [1]:
# Outside imports
import os
import importlib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [19]:
import model
import train
import evaluate
import train_util
import data_util.data
import data_util.batcher
import data_util.config
import data_util.preprocess

importlib.reload(train)
importlib.reload(model)
importlib.reload(evaluate)
importlib.reload(train_util)
importlib.reload(data_util.config)
importlib.reload(data_util.data)
importlib.reload(data_util.batcher)
importlib.reload(data_util.preprocess)

from train import *
from evaluate import *
from model import *
from train_util import *
from data_util.data import *
from data_util.batcher import *
from data_util.preprocess import *


from data_util import config
import data_util.wordVectorLoader

importlib.reload(data_util.wordVectorLoader)
from data_util import wordVectorLoader as W


In [3]:
glove_path = '/Users/rowancassius/Desktop/glove.6B/glove.6B.200d.txt'

In [4]:
vocab_path = os.path.join(config.log_root, 'data/vocab/vocab.txt')
vocab = Vocab(vocab_path)

Finished constructing vocabulary of 6639 total words. Last word added: LANGUAGE


In [5]:
loader = W.WordVectorLoader(embed_dim=200)

In [6]:
%%timeit
loader.create_embedding_matrix(embeddings_file_name=glove_path, word_to_index=vocab._word_to_id, init_random=True)

5.72 s ± 205 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
loader.embed_mat.shape

(6639, 200)

In [8]:
loader.embed_mat

array([[ 0.00613241, -0.01906412, -0.0010782 , ..., -0.00070407,
        -0.01391576,  0.00924127],
       [-0.01717561,  0.00902334,  0.00168926, ...,  0.00538988,
        -0.0113254 ,  0.00887521],
       [-0.01549181, -0.00352721,  0.01579919, ..., -0.01589783,
        -0.00294277,  0.01040599],
       ...,
       [-0.00813926,  0.01556603,  0.01726533, ...,  0.00405215,
         0.00971919, -0.00961656],
       [ 0.01794722,  0.00813187, -0.01700601, ..., -0.01975037,
         0.01934037,  0.00409161],
       [ 0.01754543,  0.01463919,  0.0099153 , ...,  0.00851383,
        -0.00192089, -0.01527785]])

In [9]:
loader.save_embed_mat(config.save_embedding_path)

model saved at: 
 data/embeddings/embedding_6639_200.tar


In [23]:
m = TaskModel(vocab.size())

In [24]:
m.load_embeddings("embedding_6639_200.tar", fix=True)

In [25]:
m

TaskModel(
  (encoder): TaskEncoder(
    (lstm): LSTM(400, 512, batch_first=True, bidirectional=True)
    (reduce_h): Linear(in_features=1024, out_features=512, bias=True)
    (reduce_c): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (enc_attention): encoder_attention(
      (W_h): Linear(in_features=1024, out_features=1024, bias=False)
      (W_s): Linear(in_features=1024, out_features=1024, bias=True)
      (v): Linear(in_features=1024, out_features=1, bias=False)
    )
    (dec_attention): decoder_attention(
      (W_prev): Linear(in_features=512, out_features=512, bias=False)
      (W_s): Linear(in_features=512, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (x_context): Linear(in_features=1224, out_features=200, bias=True)
    (lstm): LSTMCell(200, 512)
    (p_gen_linear): Linear(in_features=2760, out_features=1, bias=True)
    (V): Linear(in_features=2048, out_features=512, bias=True)
   

In [26]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [27]:
count_parameters(m)

13583384